In [ ]:
import numpy
import math
from astropy.io import fits
from astropy.table import Table
from astropy.stats import median_absolute_deviation
from matplotlib.colors import LogNorm
from matplotlib import pyplot
from astropy.modeling import models, fitting
import astropy.constants
from astropy.cosmology import FlatLambdaCDM

#set up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
main_file = fits.open("....", memmap=True)
main_table = Table(main_file[1].data)
S = num.array(main_table['.....'])    #column name


In [ ]:
# Code from Kyle


# Open the maps file
hdu = fits.open('manga-8138-12704-MAPS-HYB10-MILESHC-MASTARHC.fits.gz')

# Get the redshift (should be the same as in the DRPall and DAPall
# files)
z = hdu[0].header['SCINPVEL']/astropy.constants.c.to('km/s').value

# Select the unique bins
unique, indx = numpy.unique(hdu['BINID'].data[3,:,:].ravel(), return_index=True)
s = 1 if unique[0] == -1 else 0
unique = unique[s:]
unique_indx = indx[s:]

# Get the radius and flux measurements
r_re = hdu['SPX_ELLCOO'].data.copy()[1,:,:].ravel()[unique_indx]
flux = numpy.ma.MaskedArray(hdu['EMLINE_GFLUX'].data[23,...].copy(),
                            mask=hdu['EMLINE_GFLUX_MASK'].data[23,...] > 0).ravel()[unique_indx]

# Do the two sums
sum_flux_1re = numpy.ma.sum(flux[r_re < 1])
sum_flux_tot = numpy.ma.sum(flux)

# Calculate the luminosity distance (in Mpc)
h = 1.0
H0 = 100 * h * astropy.units.km / astropy.units.s / astropy.units.Mpc
cosmo = FlatLambdaCDM(H0=H0, Om0=0.3)
ldist = cosmo.luminosity_distance(z).value

log_Mpc_in_cm = numpy.log10(astropy.constants.pc.to('cm').value) + 6
# L (H-alpha) = 4 pi D^2 F
#   - F is in 1e-17 erg/s/cm^2
#   - D is in Mpc
#   - L (H-alpha) is in erg/s
log_halpha_luminosity_1re = numpy.log10(4*numpy.pi) + numpy.log10(sum_flux_1re) \
                                - 17 + 2*numpy.log10(ldist) + 2*log_Mpc_in_cm
# From Kennicutt & Evans (2012, ARAA, 50, 531; Table 1)
sfr_1re = numpy.power(10, log_halpha_luminosity_1re - 41.27)

log_halpha_luminosity_tot = numpy.log10(4*numpy.pi) + numpy.log10(sum_flux_tot) \
                                - 17 + 2*numpy.log10(ldist) + 2*log_Mpc_in_cm
sfr_tot = numpy.power(10, log_halpha_luminosity_tot - 41.27)

hdu = fits.open('dapall-v2_7_1-2.4.1.fits')
indx = numpy.where((hdu[1].data['PLATEIFU'] == '8138-12704')
                    & (hdu[1].data['DAPTYPE'] == 'HYB10-MILESHC-MASTARHC'))[0][0]
print(sfr_1re)
print(hdu[1].data['SFR_1RE'][indx])

print(sfr_tot)
print(hdu[1].data['SFR_TOT'][indx])

In [ ]:
# Sebastian Formula

H0= 71.0
matter = 0.27
_lambda = 0.73 

SFR= L_Ha * 0.8 * 10 ** -41  # cgs units.

